### Load data

In [1]:
import pandas as pd

data = pd.read_json("../AwareData/reddit.json")

### Preprocess

Add a column of reddit subreddit labels.

In [2]:
subreddits = data['reddit_subreddit'].unique()
label_dict = dict(zip(subreddits, range(len(subreddits))))

# label_dict = {'starbucks': 0,
#             'starbucksbaristas': 1,
#             'UPSers': 2,
#             'Lowes': 3,
#             'DollarTree': 4,
#             'WalmartEmployees': 5,
#             'McLounge': 6,
#             'Fedexers': 7,
#             'TjMaxx': 8,
#             'RiteAid': 9,
#             'walmart': 10,
#             'TalesFromYourBank': 11,
#             'KrakenSupport': 12,
#             'BestBuyWorkers': 13,
#             'Bestbuy': 14,
#             'CVS': 15,
#             'Target': 16,
#             'disney': 17,
#             'Disneyland': 17,
#             'DisneyWorld': 17,
#             'WaltDisneyWorld': 17}

data['reddit_subreddit_label'] = data['reddit_subreddit'].apply(lambda x: label_dict[x])

Add a column of indices.

In [3]:
data.loc[:, 'index'] = data.index

Remove all line breaks in reddit_text.

In [4]:
for i in range(len(data)):
     data.loc[i, 'reddit_text'] = data.loc[i, 'reddit_text'].replace('\n', ' ')

In [2]:
# data.to_csv('../data/reddit.csv')
data = pd.read_csv('../data/reddit.csv')

Make a new dataframe of submissions only.

In [3]:
subs = data.loc[data['aware_post_type']=='submission'].reset_index(drop=True)

Combine submission title and text.

In [4]:
# subs.loc[:, 'text_and_title'] = [str(subs['reddit_title'].iloc[i]) + ' [SEP] ' + str(subs['reddit_text'].iloc[i]) for i in range(len(subs))]
subs.loc[:, 'text_and_title'] = [str(subs['reddit_title'].iloc[i]) + ' ' + str(subs['reddit_text'].iloc[i]) + '#' + str(subs['reddit_subreddit'].iloc[i]) for i in range(len(subs))]

Add a column of comment indices

In [5]:
subs['comment_indices'] = [[] for _ in range(len(subs))]
for i in range(len(subs)):
    sub_index = subs.loc[i, 'index']
    next_sub_index = len(data) if i == len(subs)-1 else subs.loc[i+1,'index']
    for j in range(sub_index+1, next_sub_index):
        if data.loc[j, 'reddit_parent_id'] == subs.loc[i, 'reddit_name']:
            subs['comment_indices'].iloc[i].append(data.loc[j,'index'])

Drop those submissons with no comments

In [6]:
subs = subs.drop(subs[subs['comment_indices'].str.len()==0].index)

In [7]:
subs

,Unnamed: 0,aware_post_type,aware_created_ts,reddit_id,reddit_name,reddit_created_utc,reddit_author,reddit_text,reddit_permalink,reddit_title,reddit_url,reddit_subreddit,reddit_link_id,reddit_parent_id,reddit_submission,reddit_subreddit_label,index,text_and_title,comment_indices
0,0,submission,2023-04-02T13:58:03,129sqka,t3_129sqka,1680458283,MoodyStarGirl,That's it.,/r/starbucks/comments/129sqka/hot_chai_lattes_...,Hot chai lattes shouldn't have water,https://www.reddit.com/r/starbucks/comments/12...,starbucks,NaN,NaN,NaN,0,0,Hot chai lattes shouldn't have water That's it...,"[1, 5, 12, 13]"
1,18,submission,2023-04-02T14:07:43,129t0vd,t3_129t0vd,1680458863,kotakehoe,i had the urge to make a cute baby frapp in a ...,/r/starbucks/comments/129t0vd/my_cute_little_f...,my cute little frappucino :),https://i.redd.it/0gd8tk2zyjra1.jpg,starbucks,NaN,NaN,NaN,0,18,my cute little frappucino :) i had the urge to...,"[19, 20]"
2,26,submission,2023-04-02T14:20:39,129te77,t3_129te77,1680459639,Extension_Cricket_30,Hey guys! I was wondering what benefits people...,/r/starbucks/comments/129te77/partner_benefits...,Partner Benefits at your Starbucks?,https://www.reddit.com/r/starbucks/comments/12...,starbucks,NaN,NaN,NaN,0,26,Partner Benefits at your Starbucks? Hey guys! ...,"[27, 28, 29]"
3,30,submission,2023-04-02T14:39:11,129txfg,t3_129txfg,1680460751,ateez-lvr,did anyone notice (or is it the same for you a...,/r/starbucks/comments/129txfg/not_really_a_ran...,"not really a rant, just an observation",https://www.reddit.com/r/starbucks/comments/12...,starbucks,NaN,NaN,NaN,0,30,"not really a rant, just an observation did any...","[31, 37]"
4,38,submission,2023-04-02T14:43:36,129u231,t3_129u231,1680461016,chuchae,"hi friends, so i recently got back from a we...",/r/starbucks/comments/129u231/accidental_no_show/,accidental no show,https://www.reddit.com/r/starbucks/comments/12...,starbucks,NaN,NaN,NaN,0,38,"accidental no show hi friends, so i recently...",[39]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23263,404184,submission,2023-04-16T10:57:21,12obutt,t3_12obutt,1681657041,_TheBiggestSad_,My family and I are going to Disney World in a...,/r/WaltDisneyWorld/comments/12obutt/cheap_plac...,Cheap places to eat in/out of Disney,https://www.reddit.com/r/WaltDisneyWorld/comme...,WaltDisneyWorld,NaN,NaN,NaN,20,404184,Cheap places to eat in/out of Disney My family...,"[404185, 404186, 404187, 404188, 404189, 40419..."
23264,404248,submission,2023-04-16T10:03:41,12o9gli,t3_12o9gli,1681653821,purplechunkymonkey,My daughter and I are doing a girls trip. She ...,/r/WaltDisneyWorld/comments/12o9gli/what_to_do...,What to do outside of the parks,https://www.reddit.com/r/WaltDisneyWorld/comme...,WaltDisneyWorld,NaN,NaN,NaN,20,404248,What to do outside of the parks My daughter an...,"[404249, 404250, 404251, 404252, 404253]"
23265,404258,submission,2023-04-16T09:20:24,12o89v3,t3_12o89v3,1681651224,ol_dirty_applesauce,Shopdisney.com only has the sleeveless version...,/r/WaltDisneyWorld/comments/12o89v3/is_this_to...,Is this top offered as a t-shirt or sweatshirt...,https://i.redd.it/1bf4q46hgaua1.jpg,WaltDisneyWorld,NaN,NaN,NaN,20,404258,Is this top offered as a t-shirt or sweatshirt...,[404259]
23266,404261,submission,2023-04-16T09:18:20,12o87ug,t3_12o87ug,1681651100,Savings_Spell6563,"Upon Splash Mountain closing, I would have re-...",/r/WaltDisneyWorld/comments/12o87ug/if_you_had...,If you had the power to go back in time and ch...,https://www.reddit.com/r/WaltDisneyWorld/comme...,WaltDisneyWorld,NaN,NaN,NaN,20,404261,If you had the power to go back in time and ch...,"[404262, 404263, 404264, 404265, 404266, 40426..."


### Embedding

In [8]:
import torch

if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

In [9]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2", device)

In [32]:
vectors = model.encode(subs['text_and_title'].values.tolist(),
                       convert_to_numpy=True,
                       normalize_embeddings=True)
subs['vector'] = vectors.tolist()

### KNN

In [10]:
import lancedb

In [11]:
db = lancedb.connect("../.lancedb")
table = db.create_table("reddit", subs, mode="overwrite")
# table = db.create_table("reddit", subs, exist_ok=True)

In [34]:
# table.create_index(
#      num_partitions=256,
#      num_sub_vectors=96,
#      accelerator="mps"
# )  

In [35]:
query_string = "What is chai made of?" 
query = model.encode(query_string,
                       convert_to_numpy=True,
                       normalize_embeddings=True).tolist()

In [36]:
response = table.search(query).limit(4).to_pandas()
response

,Unnamed: 0,aware_post_type,aware_created_ts,reddit_id,reddit_name,reddit_created_utc,reddit_author,reddit_text,reddit_permalink,reddit_title,...,reddit_subreddit,reddit_link_id,reddit_parent_id,reddit_submission,reddit_subreddit_label,index,text_and_title,comment_indices,vector,_distance
0,69813,submission,2023-04-08T09:17:03,12flsj2,t3_12flsj2,1680959823,GeminiDragon60,Newbie here. How is a hot Chai latte made? Tha...,/r/starbucks/comments/12flsj2/chai_latte/,Chai latte,...,starbucks,None,None,None,0,69813,Chai latte Newbie here. How is a hot Chai latt...,[69814],"[-0.068981305, -0.057925265, -0.04843552, 0.04...",0.941060
1,235168,submission,2023-04-21T00:00:23,12tpnpv,t3_12tpnpv,1682049623,black_whitecookie23,Hey there I like the iced chai tea latte but t...,/r/starbucks/comments/12tpnpv/iced_chai_tea_la...,Iced chai tea latte,...,starbucks,None,None,None,0,235168,Iced chai tea latte Hey there I like the iced ...,"[235169, 235170, 235171, 235172]","[-0.06037707, -0.100094534, -0.040565554, -0.0...",1.122410
2,47626,submission,2023-03-31T09:00:33,127kqyb,t3_127kqyb,1680267633,rabby10,Hot or iced!,/r/starbucksbaristas/comments/127kqyb/what_is_...,What is your favorite way to enjoy a chai?,...,starbucksbaristas,None,None,None,1,47626,What is your favorite way to enjoy a chai? Hot...,"[47627, 47628, 47629, 47630, 47631, 47632, 476...","[-0.051907394, -0.015449887, -0.035115406, 0.0...",1.184249
3,74149,submission,2023-04-08T18:30:16,12g0k3f,t3_12g0k3f,1680993016,Outrageous_Ad798,I made a drink for my friend who loves gingerb...,/r/starbucks/comments/12g0k3f/my_best_creation/,My best creation,...,starbucks,None,None,None,0,74149,My best creation I made a drink for my friend ...,"[74150, 74151, 74152, 74153, 74154]","[-0.106351525, -0.0068042516, 0.008766836, 0.0...",1.214375


In [39]:
for i in range(len(response)):
    sub_index = response.loc[i, 'index']
    comment_indices = response.loc[i, 'comment_indices']
    print('Submission title: ', data.loc[sub_index, 'reddit_title'])
    print('Submission text:', data.loc[sub_index, 'reddit_text'])
    for j, k in enumerate(comment_indices):
        print('Comment', j+1, ':\n', data.loc[k, 'reddit_text'])
    # print('Comment:\n', data.loc[comment_indices[0], 'reddit_text'])
    print()

Submission title:  Chai latte
Submission text: Newbie here. How is a hot Chai latte made? Thanks!
Comment 1 :
 1. Steam 2% milk 2. Pump chai: 2/3/4/5 for short/tall/grande/venti  3. Fill cup halfway with hot water 4. Fill remaining half with steamed milk  (https://sbuxdates.com is a great site for checking recipes)

Submission title:  Iced chai tea latte
Submission text: Hey there I like the iced chai tea latte but the chai tea concentrate in stores in so expensive and barely lasts a week. Is there anyway I can make the chai tea at home? Like actual homemade chai tea with spices and tea? Also do you know what brand is the most similar to the Starbucks brand soy milk? Thanks so much!
Comment 1 :
 I mean chai tea is pretty easy to find in the grocery store, or higher end tea suppliers. Just add a fuckton of sugar 🤷‍♀️  And Silk is most similar to sbux.
Comment 2 :
 I haven't tried either but trader Joe's has a chai concrete and a chai powder
Comment 3 :
 Easy, I got you!! I’ve been makin

### Finetune the pretrained model

In [40]:
from sentence_transformers import losses
from sentence_transformers import InputExample
from torch.utils.data import DataLoader

Use (text and title, subreddit label) pairs to finetune the model, so that submissions in same subreddit are closer to each other in the embedding.

In [41]:
train_loss = losses.BatchAllTripletLoss(model=model)
train_data = [InputExample(texts=[subs['text_and_title'].iloc[i]], label=subs['reddit_subreddit_label'].iloc[i]) for i in range(len(subs))]
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=16)

In [42]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1289 [00:00<?, ?it/s]

In [43]:
model.save(path='../model', model_name='finetuned')

### KNN with the finetuned embedding

In [13]:
model_finetuned = SentenceTransformer("../model", device)

In [ ]:

vectors_finetuned = model_finetuned.encode(subs['text_and_title'].values.tolist(),
                       convert_to_numpy=True,
                       normalize_embeddings=True)
subs['vector'] = vectors_finetuned.tolist()

In [17]:
table = db.create_table("reddit_finetuned", subs, mode="overwrite")
# table = db.create_table("reddit_finedtuned", subs, exist_ok=True)

In [15]:
query_string = "What is chai made of?"
query = model.encode(query_string,
                       convert_to_numpy=True,
                       normalize_embeddings=True).tolist()

In [ ]:
response2 = table.search(query).limit(4).to_pandas()
response2

In [58]:
for i in range(len(response2)):
    sub_index = response2.loc[i, 'index']
    comment_indices = response2.loc[i, 'comment_indices']
    print('Submission title: ', data.loc[sub_index, 'reddit_title'])
    print('Submission text:', data.loc[sub_index, 'reddit_text'])
    # for j, k in enumerate(comment_indices):
    #     print('Comment', j+1, ':\n', data.loc[k, 'reddit_text'])
    print('Comment:\n', data.loc[comment_indices[0], 'reddit_text'])
    print()

Submission title:  Chap-stick
Submission text: We had a home show last weekend, and multiple people from my branch worked the booth.  On Saturday this woman came up to the booth and told us that she’s already a customer, but she was wondering if we had anymore “[Bank Name] brand chapstick”. Apparently our bank used to have chapstick tubes out in the branches for people to take for free, but they stopped doing that.  Well, this lady said that she’s been using our bank’s chapstick “every night before bed” for years and that she “can’t sleep without it.” But that now she’s run out of it and needs more. She even took my manager’s business card and has now all week called around to see if any branches have chapstick!  Well today, we finally found a little baggie of about 20 chapsticks in some random drawer of our branch. But unfortunately, they’re all expired. So my manager calls the lady and explains that we found some, but they’re no good anymore, and the bank doesn’t make them anymore.  